# Tokenizer
- HuggingFace에서 제공하는 대부분의 transformer 모델들은 텍스트를 바로 입력받을 수 없다. BERT 모델의 경우 Tokenizer로 만든 Token들을 고유한 정수로 바꾼 후 입력으로 받는다.
- HuggingFace는 각 모델별로 tokenizer를 제공하며 이는 자신의 짝인 모델이 어떤 형태의 입력을 요구하는지 알고있어 tokenizer의 출력을 모델에 넣어주기만 하면 된다.

In [2]:
from transformers import AutoTokenizer

# 해당 모델의 tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenize할 문자열을 tokenizer에 전달인자로 주기만 하면 사용할 수 있다.

In [2]:
r"""
    BatchEncoding 객체이지만 dictionary라 생각해도 무방함.
    추가적인 3 전달인자의 default 값은 True
"""
tokens = tokenizer("I love NLP!",
				   return_token_type_ids = True,
				   return_attention_mask = True,
				   add_special_tokens = True)

print(f"input ids: {tokens['input_ids']}")
print(f"token type ids: {tokens['token_type_ids']}")
print(f"attention mask: {tokens['attention_mask']}")

input ids: [101, 1045, 2293, 17953, 2361, 999, 102]
token type ids: [0, 0, 0, 0, 0, 0, 0]
attention mask: [1, 1, 1, 1, 1, 1, 1]


token id를 token으로 변환할 수 있다.

nlp 단어가 nl과 p로 나뉜 까닭은 BERT tokenizer가 [WordPiece 알고리즘](https://huggingface.co/docs/transformers/tokenizer_summary#wordpiece)에 따라 각 subword 단위로 토큰을 분리하기 때문이다.

반대로 토큰을 id로 바꿀 수도 있다.

In [3]:
converted_token = tokenizer.convert_ids_to_tokens(tokens.input_ids[0])
converted_tokens = tokenizer.convert_ids_to_tokens(tokens.input_ids)

print(converted_token)
print(converted_tokens)

print(tokenizer.convert_tokens_to_ids(converted_token))
print(tokenizer.convert_tokens_to_ids(converted_tokens))

[CLS]
['[CLS]', 'i', 'love', 'nl', '##p', '!', '[SEP]']
101
[101, 1045, 2293, 17953, 2361, 999, 102]


In [4]:
# special token을 포함해 문자열로 변환하기
encoded_tokens = tokenizer("I love NLP!")['input_ids']
print(f"Decoded >>> {tokenizer.decode(encoded_tokens)}")
print(f"Decoded skipped special tokens >>> {tokenizer.decode(encoded_tokens, skip_special_tokens = True)}")

Decoded >>> [CLS] i love nlp! [SEP]
Decoded skipped special tokens >>> i love nlp!


[Bert Special Tokens]
- padding: [PAD] (0) - tokenizeer.pad_token
- unknown: [UNK] (100) - tokenizer.unk_token
- classifier: [CLS] (101) - tokenizer.cls_token
- seperator: [SEP] (102) - tokenizer.sep_token
- mask: [MASK] (103) - tokenizer.mask_token

In [5]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[100, 102, 0, 101, 103]
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


tokenizer의 입력에 List[str] 형태의 값이 들어오면 batch 입력이라 이해한다.

이 때 BERT는 문장A와 문장B, 2개의 문장만 입력받을 수 있다.

`List[List[str]]`의 꼴일 경우 [(문장A, 문장B), (...), ...]의 꼴로 인식하여 token_type_ids가 0과 1인 것을 확인할 수 있다.

In [6]:
tokenizer([
	["I believe", "I can fly"],
	["I have a pen", "I have an apple"]
])

{'input_ids': [[101, 1045, 2903, 102, 1045, 2064, 4875, 102], [101, 1045, 2031, 1037, 7279, 102, 1045, 2031, 2019, 6207, 102]], 'token_type_ids': [[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

return_tensors 옵션을 주면 torch_tensor 형태 출력을 받을 수도 있다.

batch 입력 시 각 텍스트의 token 수가 다르다면 pytorch tensor로 출력을 받을 수 없다.

이를 해결하기 위해 최대 길이를 정하고 패딩 또는 자르기를 사용한다.

In [7]:
tokenizer(["Pen Pineapple Apple Pen", "Thats not how i love you"],
		  return_tensors = "pt",
		  max_length = 8,
		  padding = "max_length",
		  truncation = True)

{'input_ids': tensor([[  101,  7279,  7222, 23804,  6207,  7279,   102,     0],
        [  101,  2008,  2015,  2025,  2129,  1045,  2293,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

# BertModel

Bert는 다양한 task에 대해 fine-tuning하여 사용할 수 있으며 방법은 일반적으로 다음과 같다.

1. BERT 위에 해당 task를 위한 추가 layer(head)를 쌓는다.
2. 이렇게 만들어진 모델을 추가 데이터로 학습시킨다.

## BertModel

HuggingFace에서 head가 없는 순수 BERT

1. add_pooling_layer: BERT 위에 추가적으로 pooling layer를 쌓을 것인지 여부(default: True), [CLS] 토큰의 embedding만을 뽑아 linear 연산과 activation 연산을 수행하는 layer로, BERT를 이용해 classification task를 수행할 때 주로 사용한다.
2. output_hidden_states: 각 layer의 hidden states의 출력 여부(default: False)
3. output_attentions: 각 layer의 attention distribution(attention weight)의 배열 attentions의 출력 여부(default: False)

~~`Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: [...]`: bert-base-uncased의 checkpoint의 가중치와 BertModel의 가중치가 호환되지 않아 나열된 가중치들을 버렸다는 것을 알려주는 경고, cls header의 가중치이므로 무시~~

In [4]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased",
								  add_pooling_layer = False,
								  output_hidden_states = True,
								  output_attentions = True)

In [6]:
print(f"model.num_parameters(): HuggingFace의 BERT 모델은 약 108M개의 파라미터로 이루어져 있다.\n{model.num_parameters()}")
print(f"model의 구조는 다음과 같다.\n{model}")

model.num_parameters(): HuggingFace의 BERT 모델은 약 108M개의 파라미터로 이루어져 있다.
108891648
model의 구조는 다음과 같다.
BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm(

## BertModel을 사용하는 방법

일반적인 torch module(torch.nn.Module)을 쓰듯이 tokenizer로 만든 입력값을 넣어주면 된다.

In [7]:
model_input = tokenizer("I love NLP!", return_tensors = "pt")
output = model(**model_input)

print(f"model_input: {model_input}")

print(f"BertModel의 출력값에 포함된 키들: {output.keys()}")

model_input: {'input_ids': tensor([[  101,  1045,  2293, 17953,  2361,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
BertModel의 출력값에 포함된 키들: odict_keys(['last_hidden_state', 'hidden_states', 'attentions'])


- hidden_states: 각 layer의 hidden_state를 모아놓은 리스트.
- attentions: 각 layer의 attention weight를 모아놓은 리스트.

In [8]:
print(f"hidden_states 갯수: {len(output.hidden_states)}")
print(output.hidden_states[-1] == output.last_hidden_state)
print(f"hidden_state.shape: {output.hidden_states[0].shape}")

print(f"attention 갯수: {len(output.attentions)}")
print(f"attention.shape: {output.attentions[0].shape}")

hidden_states 갯수: 13
tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])
hidden_state.shape: torch.Size([1, 7, 768])
attention 갯수: 12
attention.shape: torch.Size([1, 12, 7, 7])
